In [22]:
import pandas as pd
import numpy as np
import datetime
import urllib
import os
import json
os.environ['KAKAO_KEY'] = '9c266cb030a36f29f9b6ea9875afa2d6'
os.environ['DATASEOUL_KEY'] = '504661694b79757933354551754c70'

from datetime import datetime

from collections import defaultdict

from christmas.crawl import NaverCrawler
from christmas.crawl import AddressCoordCrawler, SubwayAddressCrawler, WeatherHistoryCrawler


addr_coord_crawl = AddressCoordCrawler()
subway_addr_crawl = SubwayAddressCrawler()
weather_history_crawl = WeatherHistoryCrawler()

def get_google_map(x, y):
    return 'https://www.google.com/maps/search/?api=1&query={x},{y}'.format(x=x, y=y)

In [23]:
path = './gendata/subway/list.csv'
subway_df = pd.read_csv(path)
subway_df.head()

,name,code,eng_name,line,out_code,address,x,y
0,4·19민주묘지,4703,April 19th National Cemetery,우이신설경전철,943,NaN,127.013696,37.649531
1,가능,1907,Ganeung,01호선,109,경기 의정부시 평화로 633,127.044288,37.748396
2,가락시장,0340,Garak Market,03호선,350,서울특별시 송파구 송파대로 지하257 (가락동),127.118263,37.493099
3,가락시장,0340,Garak Market,03호선,350,서울특별시 송파구 송파대로 지하 257 (3호선 가락시장역),127.118263,37.493099
4,가락시장,2818,Garak Market,08호선,817,서울특별시 송파구 송파대로 지하257 (가락동),127.118263,37.493099


In [25]:
subway_df[subway_df.name == '양원']

,name,code,eng_name,line,out_code,address,x,y
511,양원,1204,Yangwon,경의선,K122,서울 중랑구 송림길 147,127.107946,37.606606


In [24]:
for idx, row in subway_df.iterrows():
    addr = row.address
    if not pd.isna(addr):
        continue

    station_name = '{}역'.format(row['name'])
    addr, x, y = subway_addr_crawl(station_name)
    print('searching: {} => {}'.format(station_name, addr))
    if addr:
        subway_df.at[idx, 'address'] = addr
        subway_df.at[idx, 'x'] = x
        subway_df.at[idx, 'y'] = y
        print(get_google_map(x, y))
    else:
        station_name = '{}'.format(row['name'])
        addr, x, y = subway_addr_crawl(station_name)
        print('searching: {} => {}'.format(station_name, addr))
        if not addr:
            print(row)
        subway_df.at[idx, 'address'] = addr
        subway_df.at[idx, 'x'] = x
        subway_df.at[idx, 'y'] = y
        print(get_google_map(x, y))

searching: 4·19민주묘지역 => 
searching: 4·19민주묘지 => 
name                            4·19민주묘지
code                                4703
eng_name    April 19th National Cemetery
line                             우이신설경전철
out_code                             943
address                              NaN
x                                127.014
y                                37.6495
Name: 0, dtype: object
https://www.google.com/maps/search/?api=1&query=127.0076541423881,37.648576613403726
searching: 가양역 => 
searching: 가양 => 대전 동구 가양남로 9
https://www.google.com/maps/search/?api=1&query=127.44278054902941,36.34015626695774
searching: 가오리역 => 
searching: 가오리 => 전북 부안군 상서면 부안로 2001
https://www.google.com/maps/search/?api=1&query=126.67804310278476,35.68643427916032
searching: 가재울역 => 
searching: 가재울 => 
name            가재울
code           3216
eng_name    Gajaeul
line          인천2호선
out_code       I216
address         NaN
x           126.684
y           37.4842
Name: 10, dtype: object
https://www.goo

searching: 광운대역 => 
searching: 광운대 => 서울 노원구 광운로 20
https://www.google.com/maps/search/?api=1&query=127.058270608867,37.6192404638865
searching: 광운대역 => 
searching: 광운대 => 서울 노원구 광운로 20
https://www.google.com/maps/search/?api=1&query=127.058270608867,37.6192404638865
searching: 구리역 => 
searching: 구리 => 경기 구리시 동구릉로 197
https://www.google.com/maps/search/?api=1&query=127.13204863754271,37.61942100860677
searching: 구반포역 => 
searching: 구반포 => 서울 서초구 신반포로15길 14
https://www.google.com/maps/search/?api=1&query=126.99458206883277,37.504472691051994
searching: 구일역 => 
searching: 구일 => 부산 해운대구 선수촌로 142
https://www.google.com/maps/search/?api=1&query=129.12185107543064,35.20665387800345
searching: 국제업무지구역 => 
searching: 국제업무지구 => 
name                         국제업무지구
code                           3138
eng_name    Intl. Business District
line                            인천선
out_code                       I138
address                         NaN
x                           126.631
y                 

searching: 둔촌오륜 => 
name                 둔촌오륜
code                 4137
eng_name    Dunchon Oryun
line                 09호선
out_code              937
address               NaN
x                 127.139
y                 37.5194
Name: 228, dtype: object
https://www.google.com/maps/search/?api=1&query=127.138652069629,37.5194080372613
searching: 등촌역 => 
searching: 등촌 => 
name               등촌
code             4109
eng_name    Deungchon
line             09호선
out_code          909
address           NaN
x             126.866
y             37.5507
Name: 229, dtype: object
https://www.google.com/maps/search/?api=1&query=126.854544968588,37.5438746337198
searching: 마곡나루역 => 
searching: 마곡나루 => 
name              마곡나루
code              4105
eng_name    Magongnaru
line              09호선
out_code           905
address            NaN
x              126.827
y              37.5667
Name: 236, dtype: object
https://www.google.com/maps/search/?api=1&query=126.8274226766637,37.566711300650574
searching:

searching: 상천역 => 
searching: 상천 => 
name               상천
code             1322
eng_name    Sangcheon
line              경춘선
out_code         P133
address           NaN
x             127.455
y             37.7702
Name: 357, dtype: object
https://www.google.com/maps/search/?api=1&query=127.45451723244636,37.77017996500221
searching: 상현역 => 
searching: 상현 => 경기 용인시 수지구 심곡로 16-9
https://www.google.com/maps/search/?api=1&query=127.0766129365457,37.30576479004784
searching: 새말역 => 
searching: 새말 => 
name             새말
code           4609
eng_name    sae-mal
line         의정부경전철
out_code       U119
address         NaN
x           127.064
y           37.7488
Name: 359, dtype: object
https://www.google.com/maps/search/?api=1&query=128.06717010153966,37.446660902724126
searching: 샛강역 => 
searching: 샛강 => 경기 포천시 내촌면 금강로2606번길 1
https://www.google.com/maps/search/?api=1&query=127.244653877078,37.8027418103411
searching: 서구청역 => 
searching: 서구청 => 부산 서구 구덕로 120
https://www.google.com/maps/search/?

searching: 수리산 => 
name            수리산
code           1763
eng_name    Surisan
line           04호선
out_code        445
address         NaN
x           126.926
y           37.3506
Name: 424, dtype: object
https://www.google.com/maps/search/?api=1&query=126.91404858782367,37.372832076812294
searching: 수원시청역 => 
searching: 수원시청 => 경기 수원시 팔달구 효원로 241
https://www.google.com/maps/search/?api=1&query=127.028715898311,37.263584678785
searching: 수지구청역 => 
searching: 수지구청 => 경기 용인시 수지구 포은대로 435
https://www.google.com/maps/search/?api=1&query=127.09769075010085,37.32212300327603
searching: 숭의역 => 
searching: 숭의 => 
name             숭의
code           1889
eng_name     Sungui
line            수인선
out_code       K262
address         NaN
x           126.638
y           37.4608
Name: 436, dtype: object
https://www.google.com/maps/search/?api=1&query=126.65918116591357,37.457938470337304
searching: 시민공원역 => 
searching: 시민공원 => 부산 부산진구 시민공원로 73
https://www.google.com/maps/search/?api=1&query=129.05740381

searching: 운서역 => 
searching: 운서 => 인천 중구 공항로 272
https://www.google.com/maps/search/?api=1&query=126.4330293600463,37.46747272766454
searching: 운연역 => 
searching: 운연 => 인천 남동구 음실로 19
https://www.google.com/maps/search/?api=1&query=126.766234831389,37.4419336548974
searching: 운정역 => 
searching: 운정 => 광주 북구 민주로 200
https://www.google.com/maps/search/?api=1&query=126.93875536766643,35.23652238836194
searching: 원곡역 => 
searching: 원곡 => 경기 안산시 단원구 다문화길 16
https://www.google.com/maps/search/?api=1&query=126.79054231744732,37.329625529191105
searching: 원당역 => 
searching: 원당 => 
name             원당
code           1951
eng_name    Wondang
line           03호선
out_code        316
address         NaN
x           126.843
y           37.6531
Name: 590, dtype: object
https://www.google.com/maps/search/?api=1&query=126.70364781470161,37.60426553229222
searching: 원시역 => 
searching: 원시 => 
name             원시
code           4815
eng_name      Wonsi
line            서해선
out_code        702
address       

searching: 중앙보훈병원역 => 
searching: 중앙보훈병원 => 서울 강동구 진황도로61길 53
https://www.google.com/maps/search/?api=1&query=127.147757381562,37.5293512633534
searching: 증미역 => 
searching: 증미 => 
name             증미
code           4108
eng_name    Jeungmi
line           09호선
out_code        908
address         NaN
x           126.861
y           37.5581
Name: 687, dtype: object
https://www.google.com/maps/search/?api=1&query=126.860637888459,37.5581321602446
searching: 지식정보단지역 => 
searching: 지식정보단지 => 
name          지식정보단지
code            3135
eng_name    BIT Zone
line             인천선
out_code        I135
address          NaN
x            126.646
y            37.3775
Name: 690, dtype: object
https://www.google.com/maps/search/?api=1&query=126.645861381029,37.37750721892
searching: 지제역 => 
searching: 지제 => 
name             지제
code           1723
eng_name       Jije
line           01호선
out_code       P164
address         NaN
x           127.071
y           37.0189
Name: 691, dtype: object
https://www.

In [50]:
subway_df.at[0, 'name'] = '4.19민주묘지'
subway_df.at[0, 'address'] = '서울특별시 강북구 우이동 72-186'
subway_df.at[0, 'x'] = 127.01342124243267
subway_df.at[0, 'y'] = 37.65007860418948

In [66]:
for idx, row in subway_df.iterrows():
    addr = row.address
    if (addr) == '':
        print(row)
        subway_df.at[idx, 'x'] = N
        subway_df.at[idx, 'y'] = ''

name            가재울
code           3216
eng_name    Gajaeul
line          인천2호선
out_code       I216
address            
x           126.925
y           37.1577
Name: 10, dtype: object


ValueError: could not convert string to float: 

In [43]:
subway_df = subway_df.drop('addr', axis=1)

In [47]:
addr_coord_crawl('서울특별시 강북구 우이동 72-186')

(127.01342124243267, 37.65007860418948)

In [57]:
subway_addr_crawl('지하철 화계')

('', '127.017510491529', '37.6340873974011')

In [61]:
subway_df

,name,code,eng_name,line,out_code,address,x,y
0,4.19민주묘지,4703,April 19th National Cemetery,우이신설경전철,943,서울특별시 강북구 우이동 72-186,127.013421,37.650079
1,가능,1907,Ganeung,01호선,109,경기 의정부시 평화로 633,127.044288,37.748396
2,가락시장,0340,Garak Market,03호선,350,서울특별시 송파구 송파대로 지하257 (가락동),127.118263,37.493099
3,가락시장,0340,Garak Market,03호선,350,서울특별시 송파구 송파대로 지하 257 (3호선 가락시장역),127.118263,37.493099
4,가락시장,2818,Garak Market,08호선,817,서울특별시 송파구 송파대로 지하257 (가락동),127.118263,37.493099
5,가락시장,2818,Garak Market,08호선,817,서울특별시 송파구 송파대로 지하 257 (3호선 가락시장역),127.118263,37.493099
6,가산디지털단지,1702,Gasan Digital Complex,01호선,P142,서울특별시 금천구 벚꽃로 309 (가산동),126.882662,37.480396
7,가산디지털단지,2748,Gasan Digital Complex,07호선,746,서울특별시 금천구 벚꽃로 309 (가산동),126.882662,37.480396
8,가양,4107,Gayang,09호선,907,대전 동구 가양남로 9,127.442781,36.340156
9,가오리,4704,Gaori,우이신설경전철,944,전북 부안군 상서면 부안로 2001,126.678043,35.686434
